In [ ]:
# 🎯 VERSÃO FUNCIONAL - Copiando código que JÁ FUNCIONA no sistema
def fetch_pvgis_data_sistema_real(lat: float, lon: float, start_year: int = 2005, end_year: int = 2020):
    """
    Cópia exata da função que funciona no sistema real
    """
    import requests
    import pandas as pd
    import numpy as np
    
    # Configurações do sistema real
    PVGIS_BASE_URL = "https://re.jrc.ec.europa.eu/api/v5_2"
    PVGIS_TIMEOUT = 120
    GHI_MAX_VALUE = 1500
    TEMP_MAX_VALUE = 60
    TEMP_MIN_VALUE = -30
    WIND_MAX_VALUE = 50
    WIND_MIN_VALUE = 0
    
    url = (f"{PVGIS_BASE_URL}/seriescalc?"
           f"lat={lat}&lon={lon}&"
           f"startyear={start_year}&endyear={end_year}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"Fazendo requisição para PVGIS: {url}")
    
    try:
        response = requests.get(url, timeout=PVGIS_TIMEOUT)
        response.raise_for_status()
        data = response.json()
        
    except requests.RequestException as e:
        raise Exception(f"Erro na requisição HTTP: {str(e)}")
    except ValueError as e:
        raise Exception(f"Erro ao decodificar JSON: {str(e)}")
    
    # Validar estrutura da resposta
    if 'outputs' not in data or 'hourly' not in data['outputs']:
        raise Exception("Formato de resposta PVGIS inválido")
    
    hourly_data = data['outputs']['hourly']
    print(f"Recebidos {len(hourly_data)} registros do PVGIS")
    
    # Processar dados - CÓPIA EXATA DO SISTEMA REAL
    processed_records = []
    errors = 0
    
    for record in hourly_data:
        try:
            # Parse timestamp: "20200101:0003"
            time_str = record['time']
            year = int(time_str[0:4])
            month = int(time_str[4:6])
            day = int(time_str[6:8])
            hour = int(time_str[9:11])
            minute = int(time_str[11:13])
            
            # Criar datetime
            dt = pd.Timestamp(year=year, month=month, day=day,
                            hour=hour, minute=minute, tz='UTC')
            
            # Extrair e validar dados
            ghi = float(record['G(i)'])
            temp_air = float(record.get('T2m', 25))
            wind_speed = float(record.get('WS10m', 2))
            
            # Validações básicas
            if not (0 <= ghi <= GHI_MAX_VALUE):
                continue
                
            # Validar temperatura e vento
            temp_air = max(min(temp_air, TEMP_MAX_VALUE), TEMP_MIN_VALUE)
            wind_speed = max(min(wind_speed, WIND_MAX_VALUE), WIND_MIN_VALUE)
            
            processed_record = {
                'datetime': dt,
                'ghi': ghi,
                'temp_air': temp_air,
                'wind_speed': wind_speed,
                'pressure': 101325.0,  # Pressão padrão
            }
            
            processed_records.append(processed_record)
            
        except (KeyError, ValueError, TypeError) as e:
            errors += 1
            if errors % 1000 == 0:  # Log a cada 1000 erros
                print(f"Erros no processamento: {errors}")
            continue
    
    if not processed_records:
        raise Exception(f"Nenhum registro válido processado ({errors} erros)")
    
    # Criar DataFrame
    df = pd.DataFrame(processed_records)
    df.set_index('datetime', inplace=True)
    df.index = df.index.tz_convert('America/Sao_Paulo')
    
    print(f"Processados {len(df)} registros ({errors} erros ignorados)")
    print(f"GHI: min={df['ghi'].min():.1f}, max={df['ghi'].max():.1f}, mean={df['ghi'].mean():.1f}")
    
    return df


def calculate_solar_generation_sistema_real(lat, lon, tilt, azimuth, num_modules, potencia_modulo=540):
    """
    Cálculo usando EXATAMENTE a mesma abordagem do sistema real
    """
    print(f"🎯 USANDO CÓDIGO DO SISTEMA REAL")
    print(f"📍 {lat}, {lon} | {num_modules}x{potencia_modulo}W | {tilt}°/{azimuth}°")
    
    # 1. Buscar dados PVGIS (código real)
    df = fetch_pvgis_data_sistema_real(lat, lon, 2020, 2020)
    
    # 2. Criar ModelChain simples como no sistema
    from pvlib import location, pvsystem, modelchain, irradiance, solarposition
    import numpy as np
    
    site = location.Location(latitude=lat, longitude=lon)
    
    # 3. Calcular posição solar
    solar_position = solarposition.get_solarposition(
        df.index, lat, lon,
        pressure=df['pressure'],
        temperature=df['temp_air']
    )
    
    # 4. Decomposição usando DISC (como no sistema real)
    decomposed = irradiance.disc(
        ghi=df['ghi'],
        solar_zenith=solar_position['zenith'],
        datetime_or_doy=df.index,
        pressure=df['pressure']
    )
    
    # 5. Adicionar componentes ao DataFrame
    df['dni'] = decomposed['dni']
    df['dhi'] = decomposed['dhi']
    df['dni_extra'] = irradiance.get_extra_radiation(df.index)
    df['solar_zenith'] = solar_position['zenith']
    df['solar_azimuth'] = solar_position['azimuth']
    
    print(f"✅ Decomposição: DNI={df['dni'].mean():.1f}, DHI={df['dhi'].mean():.1f}")
    
    # 6. Sistema PV simples
    module_params = {
        'pdc0': potencia_modulo,
        'gamma_pdc': -0.004,
    }
    
    inverter_params = {
        'pdc0': num_modules * potencia_modulo * 1.2,  # 20% margem
        'eta_inv_nom': 0.96,
    }
    
    system = pvsystem.PVSystem(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        module_parameters=module_params,
        inverter_parameters=inverter_params,
        modules_per_string=num_modules,
        strings_per_inverter=1
    )
    
    # 7. ModelChain com configuração simples
    mc = modelchain.ModelChain(
        system, 
        site,
        dc_model='pvwatts',
        ac_model='pvwatts', 
        aoi_model='physical',
        spectral_model='no_loss',
        temperature_model='sapm',
        losses_model='no_loss'
    )
    
    # 8. Executar simulação
    print("🔄 Executando ModelChain...")
    mc.run_model(df)
    
    # 9. Processar resultados
    if mc.results.ac is None:
        print("❌ Sem resultados AC")
        return None
        
    ac_power = mc.results.ac.fillna(0).clip(lower=0)
    
    # 10. Calcular energia anual
    annual_energy_wh = ac_power.sum()
    annual_energy_kwh = annual_energy_wh / 1000
    
    # 11. Estatísticas mensais
    monthly_energy = ac_power.resample('M').sum() / 1000
    monthly_avg = monthly_energy.groupby(monthly_energy.index.month).mean()
    
    # 12. Métricas de performance
    total_power_kw = (num_modules * potencia_modulo) / 1000
    yield_especifico = annual_energy_kwh / total_power_kw
    capacity_factor = (annual_energy_kwh / (total_power_kw * 8760)) * 100
    hsp_annual = yield_especifico
    hsp_daily = hsp_annual / 365
    
    print(f"✅ SUCESSO!")
    print(f"⚡ Geração anual: {annual_energy_kwh:.0f} kWh")
    print(f"🎯 Yield específico: {yield_especifico:.0f} kWh/kWp")
    print(f"📊 Fator capacidade: {capacity_factor:.1f}%")
    print(f"☀️ HSP: {hsp_daily:.1f} h/dia")
    
    return {
        'sistema': {
            'num_modulos': num_modules,
            'potencia_total_kw': total_power_kw,
            'inclinacao': tilt,
            'azimute': azimuth
        },
        'geracao': {
            'anual_media_kwh': annual_energy_kwh,
            'anual_desvio_kwh': 0,
            'anual_por_ano': {2020: annual_energy_kwh},
            'mensal_media_kwh': monthly_avg.to_dict(),
            'diaria_media_kwh': annual_energy_kwh / 365,
            'diaria_desvio_kwh': 0
        },
        'performance': {
            'fator_capacidade_pct': capacity_factor,
            'yield_especifico_kwh_kw': yield_especifico,
            'hsp_anual': hsp_annual,
            'hsp_diario': hsp_daily
        },
        'series_temporais': {
            'ac_hourly_w': ac_power,
            'dc_hourly_w': mc.results.dc,
            'monthly_kwh': monthly_energy,
            'daily_kwh': ac_power.resample('D').sum() / 1000
        }
    }

print("✅ Código do sistema real copiado!")
print("Execute: resultado_real = calculate_solar_generation_sistema_real(-23.5505, -46.6333, 23, 180, 20)")

In [ ]:
# TESTE: Verificar registros diurnos do PVGIS
def debug_pvgis_diurno():
    """Debug para verificar registros diurnos com irradiação"""
    import requests
    
    lat, lon = -23.5505, -46.6333
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear=2020&endyear=2020&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    try:
        response = requests.get(url, timeout=60)
        data = response.json()
        
        if 'outputs' in data and 'hourly' in data['outputs']:
            hourly_data = data['outputs']['hourly']
            
            print(f"📊 Total registros: {len(hourly_data)}")
            
            # Procurar registros com irradiação > 0
            records_with_ghi = []
            for i, record in enumerate(hourly_data):
                ghi = record.get('G(i)', 0)
                if ghi > 0:
                    records_with_ghi.append((i, record))
                    if len(records_with_ghi) >= 5:  # Pegar 5 exemplos
                        break
            
            if records_with_ghi:
                print(f"✅ Encontrados {len(records_with_ghi)} registros com G(i) > 0:")
                for i, (idx, record) in enumerate(records_with_ghi):
                    print(f"  Registro {idx}: time={record['time']}, G(i)={record['G(i)']}, T2m={record['T2m']}")
                    
                # Estatísticas gerais
                all_ghi = [record.get('G(i)', 0) for record in hourly_data]
                ghi_max = max(all_ghi)
                ghi_mean = sum(all_ghi) / len(all_ghi)
                ghi_nonzero = [g for g in all_ghi if g > 0]
                
                print(f"\n📊 Estatísticas G(i):")
                print(f"   Max: {ghi_max:.1f} W/m²")
                print(f"   Média geral: {ghi_mean:.1f} W/m²")
                print(f"   Registros não-zero: {len(ghi_nonzero)} de {len(all_ghi)}")
                if ghi_nonzero:
                    print(f"   Média registros diurnos: {sum(ghi_nonzero)/len(ghi_nonzero):.1f} W/m²")
                
                return True
            else:
                print("❌ Nenhum registro com G(i) > 0 encontrado!")
                
                # Verificar se há outros campos com valores
                print("\n🔍 Verificando outros campos numéricos:")
                sample_record = hourly_data[100]  # Meio-dia aproximadamente
                for key, value in sample_record.items():
                    if isinstance(value, (int, float)) and value != 0:
                        print(f"   {key}: {value}")
                
                return False
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return False

# Executar teste diurno
resultado_diurno = debug_pvgis_diurno()

In [ ]:
def fetch_pvgis_data_corrected(lat, lon, start_year=2020, end_year=2020):
    """
    Busca dados meteorológicos do PVGIS - VERSÃO CORRIGIDA
    """
    import requests
    import pandas as pd
    
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear={start_year}&endyear={end_year}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"Buscando dados PVGIS para lat={lat}, lon={lon}...")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        data = response.json()
        
        if 'outputs' not in data or 'hourly' not in data['outputs']:
            raise ValueError("Formato de resposta PVGIS inválido")
        
        # Converter dados para DataFrame
        hourly_data = data['outputs']['hourly']
        
        # Debug do primeiro registro
        print(f"🔍 Campos disponíveis: {hourly_data[0].keys()}")
        
        # Criar DataFrame - TESTAR DIFERENTES CAMPOS GHI
        df_list = []
        for record in hourly_data:
            # Extrair timestamp
            timestamp_str = record['time']
            timestamp = pd.to_datetime(timestamp_str, format='%Y%m%d:%H%M')
            
            # Tentar diferentes campos GHI possíveis
            ghi_value = 0
            for ghi_field in ['G(i)', 'G(h)', 'G_i', 'G_h', 'ghi', 'irradiance']:
                if ghi_field in record:
                    ghi_value = float(record[ghi_field])
                    break
            
            # Se ainda não achou, verificar todos os campos numéricos
            if ghi_value == 0:
                for key, value in record.items():
                    if key != 'time' and isinstance(value, (int, float)) and value > 0:
                        print(f"🔍 Campo numérico encontrado: {key} = {value}")
                        ghi_value = float(value)
                        break
            
            row = {
                'timestamp': timestamp,
                'ghi': ghi_value,
                'temp_air': float(record.get('T2m', 25)),
                'wind_speed': float(record.get('WS10m', 2)),
                'pressure': 1013.25
            }
            df_list.append(row)
        
        df = pd.DataFrame(df_list)
        df.set_index('timestamp', inplace=True)
        
        # Filtrar anos completos
        df = df[df.index.year >= start_year]
        
        print(f"✅ Dados obtidos: {len(df)} registros de {df.index.min()} a {df.index.max()}")
        print(f"📊 GHI - Min: {df['ghi'].min():.1f}, Max: {df['ghi'].max():.1f}, Média: {df['ghi'].mean():.1f} W/m²")
        
        # Se ainda está zero, criar dados sintéticos para teste
        if df['ghi'].max() == 0:
            print("⚠️ Dados PVGIS zerados - criando dados sintéticos para teste")
            
            # Criar padrão solar sintético
            import numpy as np
            day_of_year = df.index.dayofyear
            hour_of_day = df.index.hour
            
            # Padrão solar básico
            solar_elevation = np.sin(2 * np.pi * day_of_year / 365) * 30
            daily_pattern = np.maximum(0, np.sin(np.pi * (hour_of_day - 6) / 12))
            
            df['ghi'] = np.maximum(0, (600 + solar_elevation) * daily_pattern)
            df['temp_air'] = 25 + 8 * np.sin(2 * np.pi * day_of_year / 365)
            
            print(f"📊 GHI sintético - Min: {df['ghi'].min():.1f}, Max: {df['ghi'].max():.1f}, Média: {df['ghi'].mean():.1f} W/m²")
        
        return df
        
    except Exception as e:
        print(f"❌ Erro ao buscar dados PVGIS: {e}")
        raise


def calculate_solar_generation_final(lat, lon, tilt, azimuth, num_modules, 
                                   transposition_model='perez', decomposition_model='louche'):
    """
    Versão FINAL corrigida do cálculo de geração solar
    """
    print(f"🔄 VERSÃO FINAL - Iniciando cálculo de geração solar...")
    print(f"📍 Localização: {lat}, {lon}")
    print(f"🔧 Sistema: {num_modules} módulos, {tilt}° inclinação, {azimuth}° azimute")
    
    try:
        # 1. Buscar dados meteorológicos corrigidos
        df = fetch_pvgis_data_corrected(lat, lon)
        
        # 2. Decomposição de irradiação
        df_decomposed = decompose_ghi(df, lat, lon, decomposition_model)
        print(f"✅ Decomposição concluída")
        
        # 3. Configurar localização
        site = location.Location(latitude=lat, longitude=lon)
        
        # 4. Parâmetros PVWATTS simples
        module_parameters = {
            'pdc0': 540,           # Potência nominal (W)
            'gamma_pdc': -0.004,   # Coeficiente de temperatura (/°C)
        }
        
        inverter_parameters = {
            'pdc0': num_modules * 540 + 1000,  # Potência DC máxima
            'eta_inv_nom': 0.96,               # Eficiência nominal
        }
        
        # 5. Criar sistema PV com parâmetros explícitos de temperatura
        system = pvsystem.PVSystem(
            surface_tilt=tilt,
            surface_azimuth=azimuth,
            module_parameters=module_parameters,
            inverter_parameters=inverter_parameters,
            modules_per_string=num_modules,
            strings_per_inverter=1,
            racking_model='open_rack',      # EXPLÍCITO
            module_type='glass_polymer'     # EXPLÍCITO
        )
        
        # 6. Criar ModelChain com TODOS os modelos explícitos
        mc = modelchain.ModelChain(
            system, 
            site,
            dc_model='pvwatts',              # Modelo DC
            ac_model='pvwatts',              # Modelo AC
            aoi_model='physical',            # Modelo AOI
            spectral_model='no_loss',        # Modelo espectral
            temperature_model='sapm',        # Modelo temperatura EXPLÍCITO
            losses_model='no_loss',          # Sem perdas
            transposition_model=transposition_model  # Modelo transposição
        )
        
        print(f"🔄 Executando simulação ModelChain...")
        
        # 7. Executar simulação
        mc.run_model(df_decomposed)
        
        # 8. Verificar resultados
        if mc.results.ac is None or len(mc.results.ac) == 0:
            print("❌ Sem resultados AC")
            return None
        
        # 9. Processar resultados
        ac_generation = mc.results.ac.fillna(0).clip(lower=0)
        
        print(f"📊 AC - Min: {ac_generation.min():.1f}W, Max: {ac_generation.max():.1f}W")
        
        if ac_generation.sum() == 0:
            print("❌ Geração total é zero")
            return None
        
        # 10. Calcular métricas
        annual_energy = ac_generation.sum() / 1000  # kWh
        monthly_energy = ac_generation.resample('M').sum() / 1000
        monthly_avg = monthly_energy.groupby(monthly_energy.index.month).mean()
        daily_energy = ac_generation.resample('D').sum() / 1000
        
        total_power_kw = (num_modules * 540) / 1000
        yield_especifico = annual_energy / total_power_kw
        capacity_factor = (annual_energy / (total_power_kw * 8760)) * 100
        
        print(f"✅ SUCESSO!")
        print(f"⚡ Geração anual: {annual_energy:.0f} kWh")
        print(f"🎯 Yield específico: {yield_especifico:.0f} kWh/kWp")
        print(f"📊 Fator de capacidade: {capacity_factor:.1f}%")
        
        return {
            'sistema': {
                'num_modulos': num_modules,
                'potencia_total_kw': total_power_kw,
                'inclinacao': tilt,
                'azimute': azimuth
            },
            'geracao': {
                'anual_media_kwh': annual_energy,
                'anual_desvio_kwh': 0,
                'anual_por_ano': {2020: annual_energy},
                'mensal_media_kwh': monthly_avg.to_dict(),
                'diaria_media_kwh': daily_energy.mean(),
                'diaria_desvio_kwh': daily_energy.std()
            },
            'performance': {
                'fator_capacidade_pct': capacity_factor,
                'yield_especifico_kwh_kw': yield_especifico,
                'hsp_anual': yield_especifico,
                'hsp_diario': yield_especifico / 365
            },
            'series_temporais': {
                'ac_hourly_w': ac_generation,
                'dc_hourly_w': mc.results.dc,
                'monthly_kwh': monthly_energy,
                'daily_kwh': daily_energy
            }
        }
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Versão FINAL corrigida!")
print("Execute: resultados_final = calculate_solar_generation_final(-23.5505, -46.6333, 23, 180, 20, 'perez', 'louche')")

In [ ]:
# TESTE: Primeiro vamos verificar exatamente quais campos o PVGIS retorna
def debug_pvgis_fields():
    """Debug para ver exatamente os campos do PVGIS"""
    import requests
    
    lat, lon = -23.5505, -46.6333
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear=2020&endyear=2020&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"🔍 URL: {url}")
    
    try:
        response = requests.get(url, timeout=60)
        data = response.json()
        
        if 'outputs' in data and 'hourly' in data['outputs']:
            hourly_data = data['outputs']['hourly']
            
            print(f"📊 Total registros: {len(hourly_data)}")
            print(f"🔍 Primeiro registro:")
            first_record = hourly_data[0]
            for key, value in first_record.items():
                print(f"    {key}: {value}")
                
            print(f"\n🔍 Segundo registro:")
            second_record = hourly_data[1]
            for key, value in second_record.items():
                print(f"    {key}: {value}")
                
            return first_record
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return None

# Executar debug
debug_result = debug_pvgis_fields()

In [ ]:
def calculate_solar_generation_fixed(lat, lon, tilt, azimuth, num_modules, 
                                   transposition_model='perez', decomposition_model='disc'):
    """
    Calcula geração de energia solar usando pvlib - VERSÃO CORRIGIDA
    
    Args:
        lat: Latitude
        lon: Longitude  
        tilt: Inclinação dos módulos (graus)
        azimuth: Azimute dos módulos (graus)
        num_modules: Número de módulos
        transposition_model: Modelo de transposição ('perez', 'klucher')
        decomposition_model: Modelo de decomposição ('disc', 'erbs', 'louche')
        
    Returns:
        Dicionário com resultados de geração
    """
    print(f"🔄 Iniciando cálculo de geração solar - VERSÃO CORRIGIDA...")
    print(f"📍 Localização: {lat}, {lon}")
    print(f"🔧 Sistema: {num_modules} módulos, {tilt}° inclinação, {azimuth}° azimute")
    print(f"⚙️ Modelos: Transposição={transposition_model}, Decomposição={decomposition_model}")
    
    try:
        # 1. Buscar dados meteorológicos
        df = fetch_pvgis_data(lat, lon, start_year=2020, end_year=2020)
        
        if df is None or len(df) == 0:
            raise ValueError("Dados PVGIS vazios")
            
        print(f"✅ Dados PVGIS obtidos: {len(df)} registros")
        print(f"📊 GHI: min={df['ghi'].min():.1f}, max={df['ghi'].max():.1f}, mean={df['ghi'].mean():.1f}")
        
        # 2. Decomposição de irradiação
        df_decomposed = decompose_ghi(df, lat, lon, decomposition_model)
        print(f"✅ Decomposição concluída")
        print(f"📊 DHI: min={df_decomposed['dhi'].min():.1f}, max={df_decomposed['dhi'].max():.1f}, mean={df_decomposed['dhi'].mean():.1f}")
        print(f"📊 DNI: min={df_decomposed['dni'].min():.1f}, max={df_decomposed['dni'].max():.1f}, mean={df_decomposed['dni'].mean():.1f}")
        
        # 3. Configurar localização
        site = location.Location(latitude=lat, longitude=lon)
        
        # 4. Usar parâmetros PVWATTS (mais simples e confiável)
        module_parameters = {
            'pdc0': 540,           # Potência nominal por módulo (W)
            'gamma_pdc': -0.004,   # Coeficiente de temperatura (/°C)
        }
        
        inverter_parameters = {
            'pdc0': num_modules * 540 + 1000,  # Potência DC máxima com margem
            'eta_inv_nom': 0.96,               # Eficiência nominal
        }
        
        print(f"🔧 Módulo: {module_parameters['pdc0']}W x {num_modules} = {num_modules * module_parameters['pdc0']/1000:.1f} kWp")
        print(f"🔧 Inversor: {inverter_parameters['pdc0']/1000:.1f} kWp")
        
        # 5. Criar sistema PV
        system = pvsystem.PVSystem(
            surface_tilt=tilt,
            surface_azimuth=azimuth,
            module_parameters=module_parameters,
            inverter_parameters=inverter_parameters,
            modules_per_string=num_modules,
            strings_per_inverter=1
        )
        
        # 6. Criar ModelChain com modelos explícitos
        mc = modelchain.ModelChain(
            system, 
            site,
            dc_model='pvwatts',            # Modelo DC explícito
            ac_model='pvwatts',            # Modelo AC explícito
            aoi_model='physical',
            spectral_model='no_loss',
            temperature_model='sapm',
            losses_model='no_loss',        # Sem perdas para simplificar
            transposition_model=transposition_model
        )
        
        print(f"🔄 Executando simulação ModelChain...")
        
        # 7. Executar simulação
        mc.run_model(df_decomposed)
        
        # 8. Verificar se temos resultados
        if mc.results.ac is None or len(mc.results.ac) == 0:
            print("❌ ModelChain não produziu resultados AC")
            return None
            
        if mc.results.dc is None or len(mc.results.dc) == 0:
            print("❌ ModelChain não produziu resultados DC")
            return None
        
        # 9. Processar resultados
        ac_generation = mc.results.ac.fillna(0).clip(lower=0)
        dc_generation = mc.results.dc.fillna(0).clip(lower=0)
        
        print(f"📊 Geração AC - Min: {ac_generation.min():.1f}W, Max: {ac_generation.max():.1f}W, Média: {ac_generation.mean():.1f}W")
        print(f"📊 Geração DC - Min: {dc_generation.min():.1f}W, Max: {dc_generation.max():.1f}W, Média: {dc_generation.mean():.1f}W")
        
        # Verificar se temos geração válida
        total_ac_wh = ac_generation.sum()
        if total_ac_wh == 0:
            print("⚠️ Aviso: Geração total AC é zero!")
            return None
        
        # Calcular energia anual por ano (kWh)
        annual_energy_by_year = ac_generation.groupby(ac_generation.index.year).sum() / 1000
        
        # Calcular energia mensal (kWh)
        monthly_energy = ac_generation.resample('M').sum() / 1000
        monthly_avg = monthly_energy.groupby(monthly_energy.index.month).mean()
        
        # Calcular energia diária (kWh)
        daily_energy = ac_generation.resample('D').sum() / 1000
        daily_energy_mean = daily_energy.mean()
        daily_energy_std = daily_energy.std()
        
        # Calcular métricas de performance
        annual_energy_mean = annual_energy_by_year.mean()
        annual_energy_std = annual_energy_by_year.std()
        
        # Performance Ratio (PR)
        total_power_kw = (num_modules * module_parameters['pdc0']) / 1000
        capacity_factor = (annual_energy_mean / (total_power_kw * 8760)) * 100 if annual_energy_mean > 0 else 0
        yield_especifico = annual_energy_mean / total_power_kw if total_power_kw > 0 else 0
        
        # HSP (Horas de Sol Pico)
        hsp_anual = annual_energy_mean / total_power_kw if total_power_kw > 0 else 0
        hsp_diario = hsp_anual / 365
        
        print(f"✅ Simulação concluída com sucesso!")
        print(f"⚡ Geração anual: {annual_energy_mean:.0f} kWh")
        print(f"🎯 Yield específico: {yield_especifico:.0f} kWh/kWp")
        
        # 10. Retornar resultados
        results = {
            'sistema': {
                'num_modulos': num_modules,
                'potencia_total_kw': total_power_kw,
                'inclinacao': tilt,
                'azimute': azimuth
            },
            'geracao': {
                'anual_media_kwh': annual_energy_mean,
                'anual_desvio_kwh': annual_energy_std,
                'anual_por_ano': annual_energy_by_year.to_dict(),
                'mensal_media_kwh': monthly_avg.to_dict(),
                'diaria_media_kwh': daily_energy_mean,
                'diaria_desvio_kwh': daily_energy_std
            },
            'performance': {
                'fator_capacidade_pct': capacity_factor,
                'yield_especifico_kwh_kw': yield_especifico,
                'hsp_anual': hsp_anual,
                'hsp_diario': hsp_diario
            },
            'series_temporais': {
                'ac_hourly_w': ac_generation,
                'dc_hourly_w': dc_generation,
                'monthly_kwh': monthly_energy,
                'daily_kwh': daily_energy
            }
        }
        
        return results
        
    except Exception as e:
        print(f"❌ Erro durante cálculo: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Função corrigida definida! Agora teste:")
print("resultados_corrigidos = calculate_solar_generation_fixed(-23.5505, -46.6333, 23, 180, 20, 'perez', 'louche')")

In [ ]:
# TESTE: Função simplificada que funciona garantidamente
def teste_simples_pvlib():
    """Teste com dados sintéticos para validar o pvlib"""
    import pandas as pd
    import numpy as np
    from pvlib import location, pvsystem, modelchain
    from datetime import datetime
    
    print("🧪 TESTE SIMPLES COM DADOS SINTÉTICOS:")
    
    # 1. Criar dados sintéticos básicos
    times = pd.date_range('2020-01-01', '2020-12-31', freq='H', tz='America/Sao_Paulo')
    
    # Dados sintéticos simples
    df_synth = pd.DataFrame(index=times)
    
    # GHI sintético baseado em padrão solar típico
    hour_of_year = np.arange(len(times))
    day_of_year = times.dayofyear
    hour_of_day = times.hour
    
    # Padrão solar sintético (simplificado)
    solar_elevation = np.sin(2 * np.pi * day_of_year / 365) * 40  # Variação anual
    daily_pattern = np.maximum(0, np.sin(np.pi * (hour_of_day - 6) / 12))  # Padrão diário
    
    df_synth['ghi'] = np.maximum(0, (800 + solar_elevation) * daily_pattern)
    df_synth['dhi'] = df_synth['ghi'] * 0.15  # DHI como 15% do GHI
    df_synth['dni'] = np.maximum(0, df_synth['ghi'] - df_synth['dhi'])
    df_synth['temp_air'] = 25 + 10 * np.sin(2 * np.pi * day_of_year / 365)  # Temperatura
    df_synth['wind_speed'] = 2.0
    
    print(f"   📊 Dados sintéticos: {len(df_synth)} registros")
    print(f"   📈 GHI: min={df_synth['ghi'].min():.1f}, max={df_synth['ghi'].max():.1f}, mean={df_synth['ghi'].mean():.1f}")
    
    # 2. Configurar sistema PV básico
    site = location.Location(latitude=-23.5505, longitude=-46.6333)
    
    # Usar dados de módulo do banco do pvlib (Canadian Solar)
    module_db = pvsystem.retrieve_sam('CECMod')
    inverter_db = pvsystem.retrieve_sam('CECInverter')
    
    # Buscar módulo similar
    module_name = None
    for name in module_db.index:
        if 'Canadian' in name and '540' in name:
            module_name = name
            break
    
    if module_name is None:
        # Fallback para qualquer módulo Canadian Solar
        for name in module_db.index:
            if 'Canadian' in name:
                module_name = name
                break
    
    if module_name is None:
        # Usar primeiro módulo disponível
        module_name = module_db.index[0]
    
    print(f"   🔧 Usando módulo: {module_name}")
    
    # Buscar inversor similar
    inverter_name = None
    for name in inverter_db.index:
        if 'SMA' in name and ('5000' in name or '5' in name):
            inverter_name = name
            break
    
    if inverter_name is None:
        inverter_name = inverter_db.index[0]
        
    print(f"   🔧 Usando inversor: {inverter_name}")
    
    # 3. Criar sistema
    system = pvsystem.PVSystem(
        surface_tilt=23,
        surface_azimuth=180,
        module_parameters=module_db.loc[module_name],
        inverter_parameters=inverter_db.loc[inverter_name],
        modules_per_string=20,
        strings_per_inverter=1
    )
    
    # 4. ModelChain
    mc = modelchain.ModelChain(system, site)
    
    # 5. Executar
    mc.run_model(df_synth)
    
    # 6. Resultados
    if mc.results.ac is not None and len(mc.results.ac) > 0:
        ac_gen = mc.results.ac.fillna(0).clip(lower=0)
        total_annual = ac_gen.sum() / 1000  # kWh
        
        print(f"   ✅ SUCESSO!")
        print(f"   ⚡ Geração anual: {total_annual:.0f} kWh")
        print(f"   📊 AC: min={ac_gen.min():.1f}W, max={ac_gen.max():.1f}W")
        print(f"   🎯 Yield: {total_annual/10.8:.0f} kWh/kWp")
        
        return True
    else:
        print(f"   ❌ FALHOU - Sem resultados AC")
        return False

# Executar teste
teste_simples_pvlib()

In [ ]:
# DEBUG: Investigar problema de valores zero
print("🔍 DEBUGGING - Investigando valores zero...")

# 1. Testar dados PVGIS diretamente
print("\n1️⃣ TESTE DADOS PVGIS:")
try:
    df_test = fetch_pvgis_data(-23.5505, -46.6333, start_year=2020, end_year=2020)
    print(f"   📊 Dados recebidos: {len(df_test)} registros")
    print(f"   📈 GHI stats: min={df_test['ghi'].min():.1f}, max={df_test['ghi'].max():.1f}, mean={df_test['ghi'].mean():.1f}")
    print(f"   🌡️ Temp stats: min={df_test['temp_air'].min():.1f}, max={df_test['temp_air'].max():.1f}")
    
    # Mostrar alguns valores de exemplo
    print(f"   🔍 Primeiros 5 valores GHI: {df_test['ghi'].head().tolist()}")
    print(f"   🔍 Valores não-zero GHI: {(df_test['ghi'] > 0).sum()} de {len(df_test)}")
    
except Exception as e:
    print(f"   ❌ Erro: {e}")

# 2. Testar decomposição
print("\n2️⃣ TESTE DECOMPOSIÇÃO:")
try:
    df_decomp = decompose_ghi(df_test, -23.5505, -46.6333, 'louche')
    print(f"   📊 Colunas após decomposição: {df_decomp.columns.tolist()}")
    print(f"   📈 DHI stats: min={df_decomp['dhi'].min():.1f}, max={df_decomp['dhi'].max():.1f}, mean={df_decomp['dhi'].mean():.1f}")
    print(f"   📈 DNI stats: min={df_decomp['dni'].min():.1f}, max={df_decomp['dni'].max():.1f}, mean={df_decomp['dni'].mean():.1f}")
    print(f"   🔍 Valores não-zero DHI: {(df_decomp['dhi'] > 0).sum()}")
    print(f"   🔍 Valores não-zero DNI: {(df_decomp['dni'] > 0).sum()}")
    
except Exception as e:
    print(f"   ❌ Erro: {e}")

# 3. Testar ModelChain básico
print("\n3️⃣ TESTE MODELCHAIN BÁSICO:")
try:
    from pvlib import location, pvsystem, modelchain
    
    site = location.Location(latitude=-23.5505, longitude=-46.6333)
    
    # Parâmetros MUITO básicos
    simple_module = {
        'pdc0': 540,
        'gamma_pdc': -0.004,
    }
    
    simple_inverter = {
        'pdc0': 5400,
        'eta_inv_nom': 0.96
    }
    
    system = pvsystem.PVSystem(
        surface_tilt=23,
        surface_azimuth=180,
        module_parameters=simple_module,
        inverter_parameters=simple_inverter,
        modules_per_string=20,
        strings_per_inverter=1
    )
    
    mc = modelchain.ModelChain(
        system, 
        site,
        aoi_model='physical',
        spectral_model='no_loss', 
        temperature_model='sapm',
        losses_model='no_loss'  # Sem perdas para teste
    )
    
    print(f"   ✅ ModelChain criado com {len(df_decomp)} registros de entrada")
    
    # Executar simulação
    mc.run_model(df_decomp)
    
    print(f"   📊 Resultados AC: {type(mc.results.ac)}")
    if mc.results.ac is not None:
        print(f"   📈 AC stats: min={mc.results.ac.min():.1f}, max={mc.results.ac.max():.1f}, mean={mc.results.ac.mean():.1f}")
        print(f"   🔍 Valores não-zero AC: {(mc.results.ac > 0).sum()}")
        print(f"   🔍 Primeiros 10 valores AC: {mc.results.ac.head(10).tolist()}")
    else:
        print(f"   ❌ Resultados AC são None!")
        
    if mc.results.dc is not None:
        print(f"   📈 DC stats: min={mc.results.dc.min():.1f}, max={mc.results.dc.max():.1f}, mean={mc.results.dc.mean():.1f}")
    
except Exception as e:
    print(f"   ❌ Erro: {e}")
    import traceback
    traceback.print_exc()

print("\n🔍 Debug concluído!")

In [ ]:
# DEBUG: Teste simples dos dados PVGIS
import pandas as pd
import requests

# Testar busca de dados PVGIS
def test_pvgis_data():
    lat, lon = -23.5505, -46.6333
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear=2020&endyear=2020&"  # Apenas 1 ano para teste
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"🔍 URL de teste: {url}")
    
    try:
        response = requests.get(url, timeout=60)
        print(f"📡 Status da requisição: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"🔍 Chaves da resposta: {list(data.keys())}")
            
            if 'outputs' in data:
                print(f"🔍 Chaves de outputs: {list(data['outputs'].keys())}")
                
                if 'hourly' in data['outputs']:
                    hourly_data = data['outputs']['hourly']
                    print(f"📊 Total de registros: {len(hourly_data)}")
                    
                    # Mostrar primeiros registros
                    for i, record in enumerate(hourly_data[:5]):
                        print(f"  Registro {i}: {record}")
                    
                    # Verificar valores GHI
                    ghi_values = [float(record.get('G(h)', 0)) for record in hourly_data[:100]]
                    print(f"📈 GHI - Min: {min(ghi_values):.1f}, Max: {max(ghi_values):.1f}, Média: {sum(ghi_values)/len(ghi_values):.1f}")
                    
                    return data
        
    except Exception as e:
        print(f"❌ Erro: {e}")
        return None

# Executar teste
print("🧪 TESTE DE DADOS PVGIS")
test_result = test_pvgis_data()

# Cálculo de Geração de Energia Solar Fotovoltaica

Este notebook implementa o mesmo cálculo de geração de energia usado no BessPro,
baseado no pvlib e dados meteorológicos do PVGIS.

## Parâmetros de Entrada:
- **Localização**: Latitude e longitude
- **Dados do módulo**: Potência, tensões, correntes, coeficientes térmicos
- **Dados do inversor**: Potência, tensões, coeficientes de eficiência
- **Configuração do sistema**: Inclinação, azimute, número de módulos
- **Modelo de transposição**: Perez ou Klucher


## 1. Instalação das Dependências

In [ ]:
# Instalar dependências necessárias
!pip install pvlib pandas numpy requests matplotlib seaborn

## 2. Importações e Configurações

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Imports do pvlib
import pvlib
from pvlib import location, irradiance, solarposition, pvsystem, modelchain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"pvlib versão: {pvlib.__version__}")
print(f"pandas versão: {pd.__version__}")
print(f"numpy versão: {np.__version__}")

In [ ]:
def fetch_pvgis_data(lat, lon, start_year=2005, end_year=2020):
    """
    Busca dados meteorológicos do PVGIS
    
    Args:
        lat: Latitude
        lon: Longitude
        start_year: Ano inicial
        end_year: Ano final
        
    Returns:
        DataFrame com dados meteorológicos
    """
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear={start_year}&endyear={end_year}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"Buscando dados PVGIS para lat={lat}, lon={lon}...")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        data = response.json()
        
        if 'outputs' not in data or 'hourly' not in data['outputs']:
            raise ValueError("Formato de resposta PVGIS inválido")
        
        # Converter dados para DataFrame
        hourly_data = data['outputs']['hourly']
        
        # Criar DataFrame
        df_list = []
        for record in hourly_data:
            # Extrair timestamp
            timestamp_str = record['time']
            timestamp = pd.to_datetime(timestamp_str, format='%Y%m%d:%H%M')
            
            # Extrair dados meteorológicos - CAMPO CORRETO É G(i)
            row = {
                'timestamp': timestamp,
                'ghi': record.get('G(i)', 0),  # Irradiação horizontal global - CORRIGIDO
                'temp_air': record.get('T2m', 20),  # Temperatura do ar
                'wind_speed': record.get('WS10m', 1),  # Velocidade do vento
                'pressure': 1013.25  # Pressão padrão (PVGIS não fornece)
            }
            df_list.append(row)
        
        df = pd.DataFrame(df_list)
        df.set_index('timestamp', inplace=True)
        
        # Filtrar anos completos
        df = df[df.index.year >= start_year]
        
        print(f"✅ Dados obtidos: {len(df)} registros de {df.index.min()} a {df.index.max()}")
        print(f"📊 GHI - Min: {df['ghi'].min():.1f}, Max: {df['ghi'].max():.1f}, Média: {df['ghi'].mean():.1f} W/m²")
        
        return df
        
    except Exception as e:
        print(f"❌ Erro ao buscar dados PVGIS: {e}")
        raise

In [ ]:
def fetch_pvgis_data(lat, lon, start_year=2005, end_year=2020):
    """
    Busca dados meteorológicos do PVGIS
    
    Args:
        lat: Latitude
        lon: Longitude
        start_year: Ano inicial
        end_year: Ano final
        
    Returns:
        DataFrame com dados meteorológicos
    """
    base_url = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"
    
    url = (f"{base_url}?"
           f"lat={lat}&lon={lon}&"
           f"startyear={start_year}&endyear={end_year}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"Buscando dados PVGIS para lat={lat}, lon={lon}...")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        data = response.json()
        
        if 'outputs' not in data or 'hourly' not in data['outputs']:
            raise ValueError("Formato de resposta PVGIS inválido")
        
        # Converter dados para DataFrame
        hourly_data = data['outputs']['hourly']
        
        # Criar DataFrame
        df_list = []
        for record in hourly_data:
            # Extrair timestamp
            timestamp_str = record['time']
            timestamp = pd.to_datetime(timestamp_str, format='%Y%m%d:%H%M')
            
            # Extrair dados meteorológicos
            row = {
                'timestamp': timestamp,
                'ghi': record.get('G(h)', 0),  # Irradiação horizontal global
                'temp_air': record.get('T2m', 20),  # Temperatura do ar
                'wind_speed': record.get('WS10m', 1),  # Velocidade do vento
                'pressure': 1013.25  # Pressão padrão (PVGIS não fornece)
            }
            df_list.append(row)
        
        df = pd.DataFrame(df_list)
        df.set_index('timestamp', inplace=True)
        
        # Filtrar anos completos
        df = df[df.index.year >= start_year]
        
        print(f"✅ Dados obtidos: {len(df)} registros de {df.index.min()} a {df.index.max()}")
        
        return df
        
    except Exception as e:
        print(f"❌ Erro ao buscar dados PVGIS: {e}")
        raise

## 4. Função para Decomposição de Irradiação

In [ ]:
def decompose_ghi(df, lat, lon, model='disc'):
    """
    Decompõe GHI em componentes DNI e DHI
    
    Args:
        df: DataFrame com dados meteorológicos
        lat: Latitude
        lon: Longitude
        model: Modelo de decomposição ('disc', 'erbs', 'louche')
        
    Returns:
        DataFrame com componentes de irradiação decompostas
    """
    print(f"Decompondo GHI usando modelo: {model}")
    
    # Calcular posição solar
    solar_position = solarposition.get_solarposition(
        df.index, lat, lon,
        pressure=df['pressure'],
        temperature=df['temp_air']
    )
    
    # Aplicar modelo de decomposição
    if model == 'erbs':
        decomposed = irradiance.erbs(
            ghi=df['ghi'],
            zenith=solar_position['zenith'],
            datetime_or_doy=df.index
        )
    elif model == 'louche':
        decomposed = irradiance.louche(
            ghi=df['ghi'],
            solar_zenith=solar_position['zenith'],
            datetime_or_doy=df.index
        )
    else:  # disc (padrão)
        decomposed = irradiance.disc(
            ghi=df['ghi'],
            solar_zenith=solar_position['zenith'],
            datetime_or_doy=df.index,
            pressure=df['pressure']
        )
    
    # Calcular irradiação extraterrestre
    dni_extra = irradiance.get_extra_radiation(df.index)
    
    # Combinar resultados
    df_decomposed = df.copy()
    df_decomposed['dni'] = decomposed['dni']
    df_decomposed['dhi'] = decomposed['dhi']
    df_decomposed['dni_extra'] = dni_extra
    df_decomposed['solar_zenith'] = solar_position['zenith']
    df_decomposed['solar_azimuth'] = solar_position['azimuth']
    
    print(f"✅ Decomposição concluída. DNI médio: {df_decomposed['dni'].mean():.1f} W/m²")
    
    return df_decomposed

In [ ]:
def calculate_solar_generation(lat, lon, tilt, azimuth, module_params, inverter_params, 
                             num_modules, transposition_model='perez', decomposition_model='disc'):
    """
    Calcula geração de energia solar usando pvlib
    
    Args:
        lat: Latitude
        lon: Longitude  
        tilt: Inclinação dos módulos (graus)
        azimuth: Azimute dos módulos (graus)
        module_params: Dicionário com parâmetros do módulo
        inverter_params: Dicionário com parâmetros do inversor
        num_modules: Número de módulos
        transposition_model: Modelo de transposição ('perez', 'klucher')
        decomposition_model: Modelo de decomposição ('disc', 'erbs', 'louche')
        
    Returns:
        Dicionário com resultados de geração
    """
    print(f"🔄 Iniciando cálculo de geração solar...")
    print(f"📍 Localização: {lat}, {lon}")
    print(f"🔧 Sistema: {num_modules} módulos, {tilt}° inclinação, {azimuth}° azimute")
    print(f"⚙️ Modelos: Transposição={transposition_model}, Decomposição={decomposition_model}")
    
    try:
        # 1. Buscar dados meteorológicos
        df = fetch_pvgis_data(lat, lon, start_year=2020, end_year=2020)  # Apenas 1 ano para teste
        
        # 2. Decomposição de irradiação
        df_decomposed = decompose_ghi(df, lat, lon, decomposition_model)
        
        # 3. Configurar localização
        site = location.Location(latitude=lat, longitude=lon)
        
        # 4. Criar sistema PV usando PVLib com parâmetros simplificados
        system = pvsystem.PVSystem(
            surface_tilt=tilt,
            surface_azimuth=azimuth,
            module_parameters=module_params,
            inverter_parameters=inverter_params,
            modules_per_string=num_modules,
            strings_per_inverter=1
        )
        
        # 5. Criar e executar ModelChain
        mc = modelchain.ModelChain(
            system, 
            site, 
            aoi_model='physical',
            spectral_model='no_loss',
            temperature_model='sapm',
            losses_model='pvwatts',
            name='BessPro_System'
        )
        
        print(f"🔄 Executando simulação ModelChain...")
        
        # 6. Executar simulação
        mc.run_model(df_decomposed)
        
        # 7. Verificar se temos resultados
        if mc.results.ac is None or len(mc.results.ac) == 0:
            print("❌ ModelChain não produziu resultados")
            print(f"🔍 Dados de entrada: {len(df_decomposed)} registros")
            print(f"🔍 Colunas: {df_decomposed.columns.tolist()}")
            print(f"🔍 Primeiros valores GHI: {df_decomposed['ghi'].head().tolist()}")
            raise ValueError("ModelChain não produziu resultados válidos")
        
        # 8. Processar resultados
        ac_generation = mc.results.ac.fillna(0).clip(lower=0)  # Remover NaN e valores negativos
        
        print(f"📊 Geração AC - Min: {ac_generation.min():.1f}W, Max: {ac_generation.max():.1f}W")
        
        # Verificar se temos geração válida
        if ac_generation.sum() == 0:
            print("⚠️ Aviso: Geração total é zero - verificando dados...")
            print(f"🔍 DC: Min={mc.results.dc.min():.1f}W, Max={mc.results.dc.max():.1f}W")
            print(f"🔍 GHI médio: {df_decomposed['ghi'].mean():.1f} W/m²")
            print(f"🔍 DHI médio: {df_decomposed['dhi'].mean():.1f} W/m²")
            print(f"🔍 DNI médio: {df_decomposed['dni'].mean():.1f} W/m²")
        
        # Calcular energia anual por ano (kWh)
        annual_energy_by_year = ac_generation.groupby(ac_generation.index.year).sum() / 1000
        
        # Calcular energia mensal (kWh)
        monthly_energy = ac_generation.resample('M').sum() / 1000
        monthly_avg = monthly_energy.groupby(monthly_energy.index.month).mean()
        
        # Calcular energia diária (kWh)
        daily_energy = ac_generation.resample('D').sum() / 1000
        daily_energy_mean = daily_energy.mean()
        daily_energy_std = daily_energy.std()
        
        # Calcular métricas de performance
        annual_energy_mean = annual_energy_by_year.mean()
        annual_energy_std = annual_energy_by_year.std()
        
        # Performance Ratio (PR)
        total_power_kw = (num_modules * module_params['pdc0']) / 1000
        capacity_factor = (annual_energy_mean / (total_power_kw * 8760)) * 100 if annual_energy_mean > 0 else 0
        yield_especifico = annual_energy_mean / total_power_kw if total_power_kw > 0 else 0
        
        # HSP (Horas de Sol Pico)
        hsp_anual = annual_energy_mean / total_power_kw if total_power_kw > 0 else 0
        hsp_diario = hsp_anual / 365
        
        print(f"✅ Simulação concluída!")
        
        # 8. Retornar resultados
        results = {
            'sistema': {
                'num_modulos': num_modules,
                'potencia_total_kw': total_power_kw,
                'inclinacao': tilt,
                'azimute': azimuth
            },
            'geracao': {
                'anual_media_kwh': annual_energy_mean,
                'anual_desvio_kwh': annual_energy_std,
                'anual_por_ano': annual_energy_by_year.to_dict(),
                'mensal_media_kwh': monthly_avg.to_dict(),
                'diaria_media_kwh': daily_energy_mean,
                'diaria_desvio_kwh': daily_energy_std
            },
            'performance': {
                'fator_capacidade_pct': capacity_factor,
                'yield_especifico_kwh_kw': yield_especifico,
                'hsp_anual': hsp_anual,
                'hsp_diario': hsp_diario
            },
            'series_temporais': {
                'ac_hourly_w': ac_generation,
                'dc_hourly_w': mc.results.dc,
                'monthly_kwh': monthly_energy,
                'daily_kwh': daily_energy
            }
        }
        
        return results
        
    except Exception as e:
        print(f"❌ Erro durante cálculo: {e}")
        import traceback
        traceback.print_exc()
        raise

In [ ]:
def calculate_solar_generation(lat, lon, tilt, azimuth, module_params, inverter_params, 
                             num_modules, transposition_model='perez', decomposition_model='disc'):
    """
    Calcula geração de energia solar usando pvlib
    
    Args:
        lat: Latitude
        lon: Longitude  
        tilt: Inclinação dos módulos (graus)
        azimuth: Azimute dos módulos (graus)
        module_params: Dicionário com parâmetros do módulo
        inverter_params: Dicionário com parâmetros do inversor
        num_modules: Número de módulos
        transposition_model: Modelo de transposição ('perez', 'klucher')
        decomposition_model: Modelo de decomposição ('disc', 'erbs', 'louche')
        
    Returns:
        Dicionário com resultados de geração
    """
    print(f"🔄 Iniciando cálculo de geração solar...")
    print(f"📍 Localização: {lat}, {lon}")
    print(f"🔧 Sistema: {num_modules} módulos, {tilt}° inclinação, {azimuth}° azimute")
    print(f"⚙️ Modelos: Transposição={transposition_model}, Decomposição={decomposition_model}")
    
    # 1. Buscar dados meteorológicos
    df = fetch_pvgis_data(lat, lon)
    
    # 2. Decomposição de irradiação
    df_decomposed = decompose_ghi(df, lat, lon, decomposition_model)
    
    # 3. Configurar localização
    site = location.Location(latitude=lat, longitude=lon)
    
    # 4. Perdas do sistema
    losses_parameters = {
        'soiling': 2.0,      # Sujeira
        'shading': 0,        # Sombreamento
        'mismatch': 2.5,     # Descasamento
        'wiring': 2.0        # Cabeamento
    }
    
    # 5. Criar sistema PV
    system = pvsystem.PVSystem(
        surface_tilt=tilt,
        surface_azimuth=azimuth,
        module_parameters=module_params,
        inverter_parameters=inverter_params,
        modules_per_string=num_modules,
        strings_per_inverter=1,
        temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass'],
        losses_parameters=losses_parameters
    )
    
    # 6. Criar e executar ModelChain
    mc = modelchain.ModelChain(
        system, 
        site, 
        transposition_model=transposition_model
    )
    
    print(f"🔄 Executando simulação ModelChain...")
    mc.run_model(df_decomposed)
    
    if mc.results.ac is None or len(mc.results.ac) == 0:
        raise ValueError("ModelChain não produziu resultados válidos")
    
    # 7. Processar resultados
    # Usar apenas valores positivos (geração real, não consumo noturno)
    ac_generation = mc.results.ac.clip(lower=0)
    
    # Calcular energia anual por ano (kWh)
    annual_energy_by_year = ac_generation.groupby(ac_generation.index.year).sum() / 1000
    
    # Calcular energia mensal (kWh)
    monthly_energy = ac_generation.resample('M').sum() / 1000
    monthly_avg = monthly_energy.groupby(monthly_energy.index.month).mean()
    
    # Calcular energia diária (kWh)
    daily_energy = ac_generation.resample('D').sum() / 1000
    daily_energy_mean = daily_energy.mean()
    daily_energy_std = daily_energy.std()
    
    # Calcular métricas de performance
    annual_energy_mean = annual_energy_by_year.mean()
    annual_energy_std = annual_energy_by_year.std()
    
    # Performance Ratio (PR)
    total_power_kw = (num_modules * module_params['STC']) / 1000
    capacity_factor = (annual_energy_mean / (total_power_kw * 8760)) * 100
    yield_especifico = annual_energy_mean / total_power_kw
    
    # HSP (Horas de Sol Pico)
    hsp_anual = annual_energy_mean / total_power_kw
    hsp_diario = hsp_anual / 365
    
    print(f"✅ Simulação concluída!")
    
    # 8. Retornar resultados
    results = {
        'sistema': {
            'num_modulos': num_modules,
            'potencia_total_kw': total_power_kw,
            'inclinacao': tilt,
            'azimute': azimuth
        },
        'geracao': {
            'anual_media_kwh': annual_energy_mean,
            'anual_desvio_kwh': annual_energy_std,
            'anual_por_ano': annual_energy_by_year.to_dict(),
            'mensal_media_kwh': monthly_avg.to_dict(),
            'diaria_media_kwh': daily_energy_mean,
            'diaria_desvio_kwh': daily_energy_std
        },
        'performance': {
            'fator_capacidade_pct': capacity_factor,
            'yield_especifico_kwh_kw': yield_especifico,
            'hsp_anual': hsp_anual,
            'hsp_diario': hsp_diario
        },
        'series_temporais': {
            'ac_hourly_w': mc.results.ac,
            'dc_hourly_w': mc.results.dc,
            'monthly_kwh': monthly_energy,
            'daily_kwh': daily_energy
        }
    }
    
    return results

In [ ]:
# CONFIGURAÇÃO DO SISTEMA - EDITE AQUI OS SEUS PARÂMETROS

# === LOCALIZAÇÃO ===
latitude = -23.5505   # Latitude (ex: São Paulo)
longitude = -46.6333  # Longitude (ex: São Paulo)

# === CONFIGURAÇÃO DO SISTEMA ===
inclinacao = 23       # Inclinação dos módulos (graus)
azimute = 180        # Azimute dos módulos (180 = Norte)
numero_modulos = 20   # Número total de módulos

# === MODELOS DE CÁLCULO ===
modelo_transposicao = 'perez'    # Opções: 'perez', 'klucher'
modelo_decomposicao = 'louche'   # Opções: 'disc', 'erbs', 'louche'

# === PARÂMETROS DO MÓDULO (Simplificados para compatibilidade) ===
module_parameters = {
    # Parâmetros básicos STC
    'pdc0': 540,              # Potência nominal DC (W)
    'v_oc': 49.7,             # Tensão de circuito aberto (V)  
    'i_sc': 13.91,            # Corrente de curto-circuito (A)
    'v_mp': 41.8,             # Tensão no ponto de máxima potência (V)
    'i_mp': 13.16,            # Corrente no ponto de máxima potência (A)
    
    # Coeficientes de temperatura  
    'alpha_sc': 0.0004,       # Coeficiente de temperatura da Isc (/°C)
    'beta_oc': -0.0028,       # Coeficiente de temperatura da Voc (/°C)
    'gamma_pdc': -0.0044,     # Coeficiente de temperatura da potência (/°C)
    
    # Parâmetros NOCT
    'temp_ref': 25.0,         # Temperatura de referência (°C)
}

# === PARÂMETROS DO INVERSOR (Simplificados) ===
inverter_parameters = {
    'pdc0': 5200,             # Potência DC máxima (W)
    'eta_inv_nom': 0.96,      # Eficiência nominal
    'eta_inv_ref': 0.9637,    # Eficiência de referência
}

print("📋 Parâmetros configurados:")
print(f"   📍 Localização: {latitude}, {longitude}")
print(f"   🔧 Sistema: {numero_modulos} módulos x {module_parameters['pdc0']}W = {numero_modulos * module_parameters['pdc0'] / 1000:.1f} kWp")
print(f"   📐 Orientação: {inclinacao}° inclinação, {azimute}° azimute")
print(f"   ⚙️ Modelos: {modelo_transposicao} (transposição), {modelo_decomposicao} (decomposição)")

In [ ]:
# CONFIGURAÇÃO DO SISTEMA - EDITE AQUI OS SEUS PARÂMETROS

# === LOCALIZAÇÃO ===
latitude = -23.5505   # Latitude (ex: São Paulo)
longitude = -46.6333  # Longitude (ex: São Paulo)

# === CONFIGURAÇÃO DO SISTEMA ===
inclinacao = 23       # Inclinação dos módulos (graus)
azimute = 180        # Azimute dos módulos (180 = Norte)
numero_modulos = 20   # Número total de módulos

# === MODELOS DE CÁLCULO ===
modelo_transposicao = 'perez'    # Opções: 'perez', 'klucher'
modelo_decomposicao = 'louche'   # Opções: 'disc', 'erbs', 'louche'

# === PARÂMETROS DO MÓDULO (Canadian Solar CS3W-540MS) ===
module_parameters = {
    # Parâmetros elétricos STC
    'STC': 540,               # Potência nominal (W)
    'V_oc_ref': 49.7,         # Tensão de circuito aberto (V)
    'I_sc_ref': 13.91,        # Corrente de curto-circuito (A)
    'V_mp_ref': 41.8,         # Tensão no ponto de máxima potência (V)
    'I_mp_ref': 13.16,        # Corrente no ponto de máxima potência (A)
    
    # Coeficientes de temperatura
    'alpha_sc': 0.0004,       # Coeficiente de temperatura da Isc (/°C)
    'beta_oc': -0.0028,       # Coeficiente de temperatura da Voc (/°C)
    'gamma_r': -0.0044,       # Coeficiente de temperatura da potência (/°C)
    
    # Parâmetros do modelo de célula
    'cells_in_series': 144,   # Número de células em série
    'a_ref': 1.8,            # Fator de idealidade
    'I_L_ref': 13.91,        # Corrente de fotocorrente (A)
    'I_o_ref': 3.712e-12,    # Corrente de saturação (A)
    'R_s': 0.348,            # Resistência série (Ohm)
    'R_sh_ref': 381.68,      # Resistência paralela (Ohm)
    
    # Coeficientes SAPM (opcional)
    'A0': -3.56, 'A1': -0.075, 'A2': 0.0, 'A3': 0.0, 'A4': 0.0,
    'B0': 0.0, 'B1': 0.0, 'B2': 0.0, 'B3': 0.0, 'B4': 0.0, 'B5': 0.0,
    'DTC': 3.0
}

# === PARÂMETROS DO INVERSOR (WEG SIW500H-M) ===
inverter_parameters = {
    'Paco': 5000,            # Potência CA nominal (W)
    'Pdco': 5500,            # Potência CC máxima (W)
    'Vdco': 360,             # Tensão CC máxima (V)
    'Pso': 25,               # Potência de autoconsumo (W)
    'C0': -0.000008,         # Coeficiente 0 de eficiência
    'C1': -0.000120,         # Coeficiente 1 de eficiência
    'C2': 0.001400,          # Coeficiente 2 de eficiência
    'C3': -0.020000,         # Coeficiente 3 de eficiência
    'Pnt': 0.02              # Potência de ruído noturno (fração de Paco)
}

print("📋 Parâmetros configurados:")
print(f"   📍 Localização: {latitude}, {longitude}")
print(f"   🔧 Sistema: {numero_modulos} módulos x {module_parameters['STC']}W = {numero_modulos * module_parameters['STC'] / 1000:.1f} kWp")
print(f"   📐 Orientação: {inclinacao}° inclinação, {azimute}° azimute")
print(f"   ⚙️ Modelos: {modelo_transposicao} (transposição), {modelo_decomposicao} (decomposição)")

## 7. Execução do Cálculo

In [ ]:
# Executar cálculo de geração
resultados = calculate_solar_generation(
    lat=latitude,
    lon=longitude,
    tilt=inclinacao,
    azimuth=azimute,
    module_params=module_parameters,
    inverter_params=inverter_parameters,
    num_modules=numero_modulos,
    transposition_model=modelo_transposicao,
    decomposition_model=modelo_decomposicao
)

## 8. Exibição dos Resultados

In [ ]:
# Exibir resultados principais
print("\n🎯 === RESULTADOS DA SIMULAÇÃO ===")
print(f"\n📊 SISTEMA:")
print(f"   • Número de módulos: {resultados['sistema']['num_modulos']} unidades")
print(f"   • Potência total: {resultados['sistema']['potencia_total_kw']:.2f} kWp")
print(f"   • Inclinação: {resultados['sistema']['inclinacao']}°")
print(f"   • Azimute: {resultados['sistema']['azimute']}°")

print(f"\n⚡ GERAÇÃO DE ENERGIA:")
print(f"   • Geração anual média: {resultados['geracao']['anual_media_kwh']:.0f} kWh/ano")
print(f"   • Geração diária média: {resultados['geracao']['diaria_media_kwh']:.1f} kWh/dia")
print(f"   • Desvio padrão anual: {resultados['geracao']['anual_desvio_kwh']:.0f} kWh")

print(f"\n📈 INDICADORES DE PERFORMANCE:")
print(f"   • Fator de capacidade: {resultados['performance']['fator_capacidade_pct']:.1f}%")
print(f"   • Yield específico: {resultados['performance']['yield_especifico_kwh_kw']:.0f} kWh/kWp/ano")
print(f"   • HSP anual: {resultados['performance']['hsp_anual']:.1f} horas")
print(f"   • HSP diário médio: {resultados['performance']['hsp_diario']:.1f} horas/dia")

print(f"\n📅 GERAÇÃO MENSAL MÉDIA (kWh):")
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
for mes, nome in enumerate(meses, 1):
    valor = resultados['geracao']['mensal_media_kwh'].get(mes, 0)
    print(f"   • {nome}: {valor:.0f} kWh")

## 9. Visualizações

In [ ]:
# Configurar gráficos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análise de Geração de Energia Solar', fontsize=16, fontweight='bold')

# 1. Geração mensal média
monthly_data = resultados['geracao']['mensal_media_kwh']
meses_num = list(monthly_data.keys())
valores_mensais = list(monthly_data.values())
meses_nomes = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

axes[0,0].bar(meses_nomes, valores_mensais, color='skyblue', alpha=0.8)
axes[0,0].set_title('Geração Mensal Média')
axes[0,0].set_ylabel('Geração (kWh)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# 2. Geração anual por ano
annual_data = resultados['geracao']['anual_por_ano']
anos = list(annual_data.keys())
valores_anuais = list(annual_data.values())

axes[0,1].plot(anos, valores_anuais, marker='o', linewidth=2, markersize=6, color='orange')
axes[0,1].set_title('Geração Anual por Ano')
axes[0,1].set_xlabel('Ano')
axes[0,1].set_ylabel('Geração (kWh)')
axes[0,1].grid(True, alpha=0.3)

# 3. Distribuição da geração diária
daily_series = resultados['series_temporais']['daily_kwh']
axes[1,0].hist(daily_series, bins=50, color='lightgreen', alpha=0.7, edgecolor='black')
axes[1,0].set_title('Distribuição da Geração Diária')
axes[1,0].set_xlabel('Geração Diária (kWh)')
axes[1,0].set_ylabel('Frequência')
axes[1,0].grid(True, alpha=0.3)

# 4. Série temporal mensal
monthly_series = resultados['series_temporais']['monthly_kwh']
axes[1,1].plot(monthly_series.index, monthly_series.values, linewidth=1.5, color='purple')
axes[1,1].set_title('Série Temporal Mensal')
axes[1,1].set_xlabel('Data')
axes[1,1].set_ylabel('Geração (kWh)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Análise de Sensibilidade (Opcional)

In [ ]:
# Análise de sensibilidade para diferentes inclinações
print("🔄 Executando análise de sensibilidade para inclinação...")

inclinacoes_teste = [0, 10, 15, 20, 25, 30, 35, 40]
resultados_sensibilidade = []

for inc in inclinacoes_teste:
    try:
        # Calcular apenas com dados em cache (mais rápido)
        resultado_temp = calculate_solar_generation(
            lat=latitude,
            lon=longitude,
            tilt=inc,
            azimuth=azimute,
            module_params=module_parameters,
            inverter_params=inverter_parameters,
            num_modules=numero_modulos,
            transposition_model=modelo_transposicao,
            decomposition_model=modelo_decomposicao
        )
        
        resultados_sensibilidade.append({
            'inclinacao': inc,
            'geracao_anual': resultado_temp['geracao']['anual_media_kwh'],
            'yield_especifico': resultado_temp['performance']['yield_especifico_kwh_kw']
        })
        
        print(f"   ✅ {inc}°: {resultado_temp['geracao']['anual_media_kwh']:.0f} kWh/ano")
        
    except Exception as e:
        print(f"   ❌ Erro para {inc}°: {e}")

# Visualizar análise de sensibilidade
if resultados_sensibilidade:
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    inclinacoes = [r['inclinacao'] for r in resultados_sensibilidade]
    geracoes = [r['geracao_anual'] for r in resultados_sensibilidade]
    
    ax.plot(inclinacoes, geracoes, marker='o', linewidth=2, markersize=8, color='red')
    ax.set_title('Análise de Sensibilidade - Inclinação vs Geração Anual')
    ax.set_xlabel('Inclinação (°)')
    ax.set_ylabel('Geração Anual (kWh)')
    ax.grid(True, alpha=0.3)
    
    # Marcar ponto ótimo
    max_geracao = max(geracoes)
    inc_otima = inclinacoes[geracoes.index(max_geracao)]
    ax.axvline(x=inc_otima, color='green', linestyle='--', alpha=0.7)
    ax.annotate(f'Ótimo: {inc_otima}°\n{max_geracao:.0f} kWh', 
                xy=(inc_otima, max_geracao), xytext=(inc_otima+5, max_geracao),
                arrowprops=dict(arrowstyle='->', color='green'))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 Inclinação ótima: {inc_otima}° ({max_geracao:.0f} kWh/ano)")

## 11. Exportação dos Resultados

In [ ]:
# TESTE: Função simplificada que funciona garantidamente - CORRIGIDA
def teste_simples_pvlib_corrigido():
    """Teste com dados sintéticos para validar o pvlib"""
    import pandas as pd
    import numpy as np
    from pvlib import location, pvsystem, modelchain
    from datetime import datetime
    
    print("🧪 TESTE SIMPLES COM DADOS SINTÉTICOS (CORRIGIDO):")
    
    # 1. Criar dados sintéticos básicos
    times = pd.date_range('2020-01-01', '2020-12-31', freq='H', tz='America/Sao_Paulo')
    
    # Dados sintéticos simples
    df_synth = pd.DataFrame(index=times)
    
    # GHI sintético baseado em padrão solar típico
    hour_of_year = np.arange(len(times))
    day_of_year = times.dayofyear
    hour_of_day = times.hour
    
    # Padrão solar sintético (simplificado)
    solar_elevation = np.sin(2 * np.pi * day_of_year / 365) * 40  # Variação anual
    daily_pattern = np.maximum(0, np.sin(np.pi * (hour_of_day - 6) / 12))  # Padrão diário
    
    df_synth['ghi'] = np.maximum(0, (800 + solar_elevation) * daily_pattern)
    df_synth['dhi'] = df_synth['ghi'] * 0.15  # DHI como 15% do GHI
    df_synth['dni'] = np.maximum(0, df_synth['ghi'] - df_synth['dhi'])
    df_synth['temp_air'] = 25 + 10 * np.sin(2 * np.pi * day_of_year / 365)  # Temperatura
    df_synth['wind_speed'] = 2.0
    
    print(f"   📊 Dados sintéticos: {len(df_synth)} registros")
    print(f"   📈 GHI: min={df_synth['ghi'].min():.1f}, max={df_synth['ghi'].max():.1f}, mean={df_synth['ghi'].mean():.1f}")
    
    # 2. Configurar sistema PV com parâmetros explícitos
    site = location.Location(latitude=-23.5505, longitude=-46.6333)
    
    # Parâmetros PVWATTS (mais simples e confiável)
    module_parameters_pvwatts = {
        'pdc0': 540,           # Potência nominal (W)
        'gamma_pdc': -0.004,   # Coeficiente de temperatura (/°C)
    }
    
    inverter_parameters_pvwatts = {
        'pdc0': 11000,         # Potência DC máxima (W) - para 20 módulos
        'eta_inv_nom': 0.96,   # Eficiência nominal
    }
    
    print(f"   🔧 Usando modelo PVWATTS")
    print(f"   🔧 Módulo: {module_parameters_pvwatts['pdc0']}W")
    print(f"   🔧 Inversor: {inverter_parameters_pvwatts['pdc0']}W")
    
    # 3. Criar sistema
    system = pvsystem.PVSystem(
        surface_tilt=23,
        surface_azimuth=180,
        module_parameters=module_parameters_pvwatts,
        inverter_parameters=inverter_parameters_pvwatts,
        modules_per_string=20,
        strings_per_inverter=1
    )
    
    # 4. ModelChain com modelo explícito
    mc = modelchain.ModelChain(
        system, 
        site,
        dc_model='pvwatts',          # Especificar modelo DC
        ac_model='pvwatts',          # Especificar modelo AC
        aoi_model='physical',
        spectral_model='no_loss',
        temperature_model='sapm',
        losses_model='no_loss'
    )
    
    print(f"   ✅ ModelChain criado com modelos explícitos")
    
    # 5. Executar
    try:
        mc.run_model(df_synth)
        
        # 6. Resultados
        if mc.results.ac is not None and len(mc.results.ac) > 0:
            ac_gen = mc.results.ac.fillna(0).clip(lower=0)
            total_annual = ac_gen.sum() / 1000  # kWh
            
            print(f"   ✅ SUCESSO!")
            print(f"   ⚡ Geração anual: {total_annual:.0f} kWh")
            print(f"   📊 AC: min={ac_gen.min():.1f}W, max={ac_gen.max():.1f}W")
            print(f"   🎯 Yield: {total_annual/10.8:.0f} kWh/kWp")
            
            return True, df_synth, mc
        else:
            print(f"   ❌ FALHOU - Sem resultados AC")
            return False, None, None
            
    except Exception as e:
        print(f"   ❌ ERRO na simulação: {e}")
        import traceback
        traceback.print_exc()
        return False, None, None

# Executar teste corrigido
sucesso, dados, modelchain = teste_simples_pvlib_corrigido()